In [1]:
import os 
from typing import Annotated
from tavily import TavilyClient 
from autogen import AssistantAgent,UserProxyAgent,config_list_from_json,register_function
from autogen.cache import Cache
from autogen.agentchat.contrib.capabilities import teachability
from autogen.coding import DockerCommandLineCodeExecutor,LocalCommandLineCodeExecutor

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line

In [ ]:
config_list = [
    {
    }
]
llm_config = {
    "timeout": 600,
    "cache_seed": 42,  # Use 'cache_seed' instead of 'use_cache' (True becomes a seed value)
    "config_list": config_list,
}

In [3]:
tavily = TavilyClient(api_key="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")


In [4]:
def search_tool(query : Annotated[str, "search"]) -> Annotated[str, "search results"]:
    """
    Search the web for a given query.
    """
    return tavily.get_search_context(query=query,search_depth="advanced")

In [5]:
ReAct_prompt = """
Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
"""


In [ ]:
# Define the ReAct prompt message. Assuming a "question" field is present in the context
def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

## Construct agents and initiate agent chats


In [7]:

os.makedirs("coding", exist_ok=True)

In [8]:
code_executor = LocalCommandLineCodeExecutor(work_dir="coding")

In [9]:
user_proxy = UserProxyAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config={"executor": code_executor},
)

In [10]:
assistant = AssistantAgent(
    name="Assistant",
    system_message="Only use the tools you have been provided with. Reply TERMINATE when the task is done.",
    llm_config={"config_list": config_list, "cache_seed": None},
)

In [11]:
register_function(
    search_tool,
    caller=assistant,
    executor=user_proxy,
    name="search_tool",
    description="Search the web for the given query",
)

In [12]:
# Cache LLM responses. To get different responses, change the cache_seed value.
with Cache.disk(cache_seed=43) as cache:
    user_proxy.initiate_chat(
        assistant,
        message=react_prompt_message,
        question="What is the result of super bowl 2024?",
        cache=cache,
    )

User (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: What is the result of super bowl 2024?


--------------------------------------------------------------------------------
Assistant (to User):

***** Suggested tool call (call_vdU7NR0O90ZcoLvD6MmN97tt): search_tool *****
Arguments: 
{"query":"super bowl 2024 result"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXEC

## ReAct with memory module enabled via teachability

In [13]:
# Instantiate the Teachability capability. Its parameters are all optional.
teachability = teachability.Teachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./tmp/notebook/teachability_db",
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 6



CLEARING MEMORY


In [14]:
# Now add the Teachability capability to the agent.
teachability.add_to_agent(assistant)

In [15]:

with Cache.disk(cache_seed=44) as cache:
    user_proxy.initiate_chat(
        assistant,
        message=react_prompt_message,
        question="What is the result of super bowl 2024?",
        cache=cache,
    )

User (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: What is the result of super bowl 2024?


--------------------------------------------------------------------------------


C:\Users\dhanu\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [03:32<00:00, 391kiB/s] 


Assistant (to User):

***** Suggested tool call (call_srFqXzevIhau4blg7F9NMktG): search_tool *****
Arguments: 
{"query":"Super Bowl 2024 result"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (3b30bae5-b9e6-424e-9f97-f1d9e5a8b02c): User requested to end the conversation
